# 雑に拡張したBradley-TerryモデルとSinkhorn距離に基づいた微分可能なソート・ランキングを組合せてMリーガーの強さを推定する
<span style="color:red">**注意**</span>：このノートブックで示されるMリーガーの強さは、タイトルにもある通り、（雑に拡張した）**Bradley-Terryモデル with Sinkhorn距離に基づいた微分可能なソート・ランキングで推定**されたものです。
この記事で示された強さは、あくまで推定したものであり、また、モデルが異なれば結果も変わる、ということにご注意ください。

In [1]:
import torch
from torch.optim import Adam, Adagrad
import numpy as np
from models_pytorch import BT, SinkhornRankSort, SinkhornBT
from load_dataset import remove_ranking_point, add_ranking_point, load_mleague_dataset
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
X, y = load_mleague_dataset("./data/")
y_original = remove_ranking_point(y)

print(X)
print(y)
players_all = np.array(np.unique(X))
n_players = len(players_all)
print(y_original)

[['瑞原明奈' '魚谷侑未' '日向藍子' '内川幸太郎']
 ['沢崎誠' '滝沢和典' '瀬戸熊直樹' '多井隆晴']
 ['滝沢和典' '多井隆晴' '園田賢' '佐々木寿人']
 ...
 ['瀬戸熊直樹' '松本吉弘' '勝又健志' '前原雄大']
 ['白鳥翔' '内川幸太郎' '小林剛' '瀬戸熊直樹']
 ['朝倉康心' '前原雄大' '多井隆晴' '村上淳']]
[[ 77.2  10.1 -26.  -61.3]
 [ 62.9  20.9 -21.8 -62. ]
 [ 61.9   5.8 -23.6 -44.1]
 ...
 [ 61.2   7.6 -18.2 -50.6]
 [ 60.4   3.4 -20.  -43.8]
 [ 56.   13.9 -23.1 -46.8]]
[[ 32.2   5.1 -11.  -26.3]
 [ 17.9  15.9  -6.8 -27. ]
 [ 16.9   0.8  -8.6  -9.1]
 ...
 [ 16.2   2.6  -3.2 -15.6]
 [ 15.4  -1.6  -5.   -8.8]
 [ 11.    8.9  -8.1 -11.8]]


In [3]:
enc = LabelEncoder()
enc.fit(players_all)
X_enc = enc.transform(X.ravel()).reshape(-1, 4)
dataset = torch.utils.data.TensorDataset(
    torch.tensor(X_enc), torch.tensor(y)
)
loader = torch.utils.data.DataLoader(dataset, batch_size=len(X)//10, shuffle=True)


In [4]:
bt = SinkhornBT(n_players, tau=0.01, max_iter=1000, eps=1e-3, tol=1e-2).to(device)
optim = Adam(bt.parameters())

for it in range(1000):
    running_loss = 0.0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optim.zero_grad()
        y_pred = bt(X_batch)
        loss = 0.5 * torch.mean((y_batch - y_pred)**2)
        loss.backward()
        optim.step()
        running_loss += loss.item()
    if it % 10 == 0:
        print(f"Epoch: {it+1} Loss: {running_loss / len(loader)}")

Epoch: 1 Loss: 1394.5962815285204
Epoch: 11 Loss: 1384.0828202842795
Epoch: 21 Loss: 1377.3745730760188
Epoch: 31 Loss: 1368.4159605309364
Epoch: 41 Loss: 1366.8600612366138
Epoch: 51 Loss: 1365.9220658464917
Epoch: 61 Loss: 1365.2145007797747
Epoch: 71 Loss: 1364.340403904369
Epoch: 81 Loss: 1360.4136176913385
Epoch: 91 Loss: 1358.4979897898168
Epoch: 101 Loss: 1357.5646451395116
Epoch: 111 Loss: 1357.00797301735
Epoch: 121 Loss: 1354.9386893206288
Epoch: 131 Loss: 1352.7064960552314
Epoch: 141 Loss: 1353.0249972100855
Epoch: 151 Loss: 1351.1065066809317
Epoch: 161 Loss: 1350.3986372107981
Epoch: 171 Loss: 1350.1797276714149
Epoch: 181 Loss: 1349.5230332304243
Epoch: 191 Loss: 1349.0200114393222
Epoch: 201 Loss: 1349.0227681416336
Epoch: 211 Loss: 1348.3698538619133
Epoch: 221 Loss: 1347.743069229665
Epoch: 231 Loss: 1348.9166965281752
Epoch: 241 Loss: 1348.1166496062492
Epoch: 251 Loss: 1347.7312333102561
Epoch: 261 Loss: 1347.397163495268
Epoch: 271 Loss: 1347.1680812010613
Epoch: 2

In [5]:
coef_ = bt.bt.embed.weight.detach().cpu().numpy().ravel()
strength_sorted = np.sort(coef_)[::-1]
players_sorted = enc.inverse_transform(np.argsort(coef_)[::-1])
print(players_sorted)
print(strength_sorted)

['堀慎吾' '近藤誠一' '滝沢和典' '内川幸太郎' '白鳥翔' '萩原聖人' '瀬戸熊直樹' '岡田紗佳' '勝又健志' '瑞原明奈'
 '和久津晶' '園田賢' '村上淳' '沢崎誠' '石橋伸洋' '佐々木寿人' '多井隆晴' '朝倉康心' '丸山奏子' '日向藍子'
 '鈴木たろう' '二階堂亜樹' '小林剛' '前原雄大' '黒沢咲' '魚谷侑未' '高宮まり' '松本吉弘' '藤崎智' '茅森早香']
[ 0.52791023  0.5208337   0.49293813  0.39422238  0.34028548  0.3053928
  0.2731232   0.26289013  0.22842777  0.12351097  0.11227185  0.11184385
  0.10656966  0.04033573  0.03492783  0.01533102 -0.02028644 -0.08949254
 -0.29182988 -0.36775672 -0.39023915 -0.42291704 -0.42621672 -0.62470376
 -0.686216   -0.76266867 -0.84881216 -0.94601184 -0.9569087  -1.5796827 ]
